In [2]:
!pip install transformers datasets evaluate
! pip uninstall -y transformers accelerate
! pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
import pandas as pd
def load_data(csv):
  # Reads the raw csv file and split into
  # sentences (x) and target (y)
  df = pd.read_csv(csv)  
  return df
data = load_data(f'{PATH}/data.csv')
data=data.rename(columns = {'type':'labels'})
print(data)

                                                      url  \
0       https://en.wikipedia.org/wiki/Sexhow%20railway...   
1             https://en.wikipedia.org/wiki/Eti%C3%A4inen   
2       https://en.wikipedia.org/wiki/Inverse%20functi...   
3       https://en.wikipedia.org/wiki/Stepping%20on%20...   
4             https://en.wikipedia.org/wiki/Rob%20Bradley   
...                                                   ...   
299995        https://en.wikipedia.org/wiki/Randy%20Borum   
299996  https://en.wikipedia.org/wiki/Sa%27och%20language   
299997    https://en.wikipedia.org/wiki/Philip%20Hanawalt   
299998  https://en.wikipedia.org/wiki/Vossius%20Gymnasium   
299999    https://en.wikipedia.org/wiki/Simone%20Stratigo   

                           title  \
0         Sexhow railway station   
1                       Etiäinen   
2       Inverse function theorem   
3              Stepping on Roses   
4                    Rob Bradley   
...                          ...   
299995         

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["intro"], truncation=True)

In [8]:
from datasets import Dataset
import numpy as np
train, validate, test = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.6*len(data)), int(.8*len(data))])
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
tokenized_df_train = train.map(preprocess_function, batched=True)
tokenized_df_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/180000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
id2label = {0: "HUMAN", 1: "MACHINE"}
label2id = {"HUMAN": 0, "MACHINE": 1}

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [2]:
training_args = TrainingArguments(
    output_dir="output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
   # label_names = "labels",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df_train,
    eval_dataset=tokenized_df_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

NameError: ignored

In [ ]:
!pip show datasets

Name: datasets
Version: 2.12.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, responses, tqdm, xxhash
Required-by: evaluate


In [3]:
# get the training and validation loss and accuracy values from the trainer object
train_losses = trainer.history['train_loss']
train_accs = trainer.history['train_acc']
val_losses = trainer.history['eval_loss']
val_accs = trainer.history['eval_acc']

# create a TrainingArguments object and use it to plot the metrics
training_args = TrainingArguments(output_dir='./results')
training_args.plotting_args = {'title': 'Training and Validation loss/accuracy'}

TrainingArguments.plot(
    [train_losses, val_losses],
    xaxis="epoch",
    yaxis="loss",
    metric_name="Loss",
)

TrainingArguments.plot(
    [train_accs, val_accs],
    xaxis="epoch",
    yaxis="accuracy",
    metric_name="Accuracy",
)

NameError: ignored